# HuBMAP example

In [ ]:
!uv pip install miqc_py
!uv pip install 'scanpy @ git+https://github.com/keller-mark/scanpy@af55e9d'
!uv pip install 'pylemur @ git+https://github.com/keller-mark/pyLemur@2f484b0'

In [ ]:
from compasce import run_all, create_dask_client, normalize_basic, normalize_pearson_residuals, densmap, COMPASCE_KEY, compute_diffexp, compute_lemur
from compasce.io import ComparativeData

In [ ]:
from anndata import read_zarr, read_h5ad
import numpy as np
import pandas as pd

In [ ]:
import miqc_py
import scanpy as sc

In [ ]:
adata = read_h5ad("./data/hubmap/HT_raw.h5ad")

In [ ]:
adata

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var.hugo_symbol.str.startswith("MT-")
# Genes without HUGO symbol could be NaN but we need them to be True/False
adata.var["mt"] = adata.var["mt"].apply(lambda v: v if pd.notna(v) else False)
# ribosomal genes
#adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
#adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True, log1p=False)

In [ ]:
adata

In [ ]:
np.sum(~adata.obs['pct_counts_mt'].notna())

In [ ]:
adata.obs['pct_counts_mt'] = adata.obs['pct_counts_mt'].apply(lambda v: v if pd.notna(v) else 0) 
np.sum(~adata.obs['pct_counts_mt'].notna())

In [ ]:
miqc_py.calculate_miqc(adata, detected="n_genes_by_counts", subsets_mito_percent="pct_counts_mt")

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows();

In [ ]:
alt.Chart(adata.obs.iloc[range(15000)]).mark_circle().encode(
    x="n_genes_by_counts:Q",
    y="pct_counts_mt:Q",
    color="prob_compromised:Q"
)

In [ ]:
# Filter cells
miqc_py.filter_miqc(adata)

In [ ]:
# Filter genes without Hugo symbols
adata = adata[:, adata.var['hugo_symbol'].notna()]

In [ ]:
adata.obs['age_group'] = adata.obs['age'].apply(lambda num_years: "lt_50" if num_years < 50 else "gte_50")

In [ ]:
adata

In [ ]:
adata.write_h5ad("./data/hubmap/HT_filtered.h5ad")

In [ ]:
#adata.obs['age'].hist()

In [ ]:
def get_adata():
    adata = read_h5ad("./data/hubmap/HT_filtered.h5ad")

    should_subset = True
    if should_subset:
        # subset using random sample so that multiple sample groups are represented to enable comparison
        np.random.seed(1)
        obs_subset = np.random.choice(adata.obs.index.tolist(), size=50_000, replace=False).tolist()
        var_slice = slice(0, 10_000)
        adata = adata[obs_subset, var_slice].copy()
 
    adata.layers["counts"] = adata.X.todense()
    return adata

In [ ]:
sample_id_col = "hubmap_id"
sample_group_pairs = [
  ('sex', ('Male', 'Female')),
  ('age_group', ('lt_50', 'gte_50')),
]

In [ ]:
zarr_path="data/hubmap/HT.cdata.zarr"
client=create_dask_client(memory_limit='2GB')

adata = get_adata()

if "counts" not in adata.layers:
    raise ValueError("adata.layers['counts'] must exist")
if adata.shape[0] < 1:
    raise ValueError("adata must have at least one row")
if adata.shape[1] < 1:
    raise ValueError("adata must have at least one column")

cdata = ComparativeData(
    zarr_path=zarr_path,
    sample_group_pairs=sample_group_pairs,
    sample_id_col=sample_id_col
)

adata.uns[COMPASCE_KEY] = {
    "obsType": "cell",
    "featureType": "gene",
    "featureValueType": "expression",
}
ladata = cdata.create_lazy_anndata(adata, client=client)
cdata.create_sample_anndata(ladata)

del adata

# depends on: uns/write_metadata/layers/counts
# creates: uns/write_metadata/layers/logcounts

normalize_basic(ladata)

del ladata.layers["counts"]


# depends on: uns/write_metadata/layers/counts
# creates: /layers/pearson_residuals

normalize_pearson_residuals(ladata)

del ladata.layers["logcounts"]

densmap(ladata)

# depends on: uns/write_metadata/layers/logcounts
# creates: varm/DE_cell_type_vs_rest
compute_diffexp(cdata, ladata, cell_type_col="predicted_label")

compute_lemur(cdata, ladata)